In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/mistral

Mounted at /content/drive
/content/drive/MyDrive/mistral


In [2]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install bitsandbytes accelerate xformers einops
!pip install datasets

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

from datasets import load_dataset
raw_datasets = load_dataset("orkg/SciQA")
print(raw_datasets)

model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = transformers.BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
model_id,
trust_remote_code=True,
quantization_config=bnb_config,
device_map='auto',
)

tokenizer = transformers.AutoTokenizer.from_pretrained(
model_id,
)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121

Generating train split:   0%|          | 0/1795 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/513 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 1795
    })
    validation: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 257
    })
    test: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 513
    })
})


...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [3]:
import json

def save_json(filename,data):
    with open(filename, "w", encoding="utf-8") as json_file:
        print(json.dumps(data), file=json_file)


def load_json(file__name):
    try:
        data_file = open(file__name, "r", encoding='utf-8')
        file_data = json.loads(data_file.read())
        data_file.close()
        return file_data
    except FileNotFoundError:
        return None


def main(filename="mistral_test_3_shots.json", data_part="T01"):
    total_data = load_json(filename)
    if total_data is None:
        return

    data = total_data.get(data_part)

    query_list = data["questions"]
    suggestions = data["suggestions"]
    gs = data["generated_sparql"]
    sparql = data["sparql"]

    print(len(query_list))

    q_list = query_list

    for question in q_list[len(gs):]:
        print(len(gs))
        encodeds = tokenizer(question, return_tensors="pt", add_special_tokens=False)
        model_inputs = encodeds.to('cuda')

        generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
        decoded = tokenizer.batch_decode(generated_ids)
        print(decoded[0])

        gs.append(decoded[0])
        result = {"questions": query_list, "sparql": sparql, "generated_sparql": gs, "suggestions": suggestions}
        total_data[data_part] = result
        save_json(filename, total_data)

In [ ]:
# main(data_part="T06")

In [ ]:
# main(data_part="T07")

In [4]:
main(data_part="T08")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


513
67


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]input (English text): Can you list benchmarked problems in the area of Semantic Web?
 output (Sparql query): [/INST]SELECT DISTINCT ?problem ?problem_lbl WHERE { ?rf a orkgc:ResearchField; rdfs:label ?rf_label. FILTER (str( ?rf_label) = "Semantic Web") ?paper orkgp:P30 ?rf; orkgp:P31 ?cont. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:P32 ?problem. ?problem rdfs:label ?problem_lbl. } </s><s> [INST]input (English text): What is the name of the top performing model in terms of ROUGE-2 score when benchmarked on the CL-SciSumm dataset?
 output (Sparql query): [/INST]```sql
SELECT ?top_model_name
WHERE {
  ?model orkgp:HAS_BENCHMARK ?benchmark
  FILTER (?benchmark in (SELECT orkgp:BENCHMARK FROM OrgKnowledgeGraph WHERE rdfs:label "CL-SciSumm"))
  SERVICE {
    bdbsvc:service_url bdbsvc:service;
    bdbsvc:service_param bdbsvc:query '{ "query": "{ "match": { "rouge_score": { "$gt": ?benchmark_score } } } }';
    bdbsvc:service_param bdbsvc:headers '{"Authorization": "Bearer 'https:/

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

<s> [INST]input (English text): Can you list benchmarked problems in the area of Semantic Web?
 output (Sparql query): [/INST]SELECT DISTINCT ?problem ?problem_lbl WHERE { ?rf a orkgc:ResearchField; rdfs:label ?rf_label. FILTER (str( ?rf_label) = "Semantic Web") ?paper orkgp:P30 ?rf; orkgp:P31 ?cont. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:P32 ?problem. ?problem rdfs:label ?problem_lbl. } </s><s> [INST]input (English text): Which model has achieved the highest Score score on the Atari 2600 Q*Bert benchmark dataset?
 output (Sparql query): [/INST]SELECT ?name ?score WHERE { ?model a orkgc:ResearchItem; orkgc:P20 ?name; orkgc:P21 ?score; orkgc:P30 "Atari 2600 Q*Bert"; orkgc:P32 "score". FILTER (str( ?model) regex "(Q*Bert )*$") } 
ORDER BY DESC(?score) LIMIT 1</s>
510


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]input (English text): Can you list benchmarked problems in the area of Semantic Web?
 output (Sparql query): [/INST]SELECT DISTINCT ?problem ?problem_lbl WHERE { ?rf a orkgc:ResearchField; rdfs:label ?rf_label. FILTER (str( ?rf_label) = "Semantic Web") ?paper orkgp:P30 ?rf; orkgp:P31 ?cont. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:P32 ?problem. ?problem rdfs:label ?problem_lbl. } </s><s> [INST]input (English text): What is the top benchmark score and its metric on the Words in Context dataset?
 output (Sparql query): [/INST]SELECT MAX(score) AS metric, max(score) AS best_score FROM wikitalent:DatasetMetrics WHERE wikitalent:datasetURI="https://www.example.com/WordsInContext"</s>
511


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]input (English text): Can you list benchmarked problems in the area of Semantic Web?
 output (Sparql query): [/INST]SELECT DISTINCT ?problem ?problem_lbl WHERE { ?rf a orkgc:ResearchField; rdfs:label ?rf_label. FILTER (str( ?rf_label) = "Semantic Web") ?paper orkgp:P30 ?rf; orkgp:P31 ?cont. ?cont orkgp:HAS_BENCHMARK ?benchmark; orkgp:P32 ?problem. ?problem rdfs:label ?problem_lbl. } </s><s> [INST]input (English text): List the datasets benchmarked under the Common Sense Reasoning research problem?
 output (Sparql query): [/INST]SELECT DISTINCT ?problem ?problem_label WHERE { ?paper orkgc:ResearchField; rdfs:label ?rf_label. FILTER (str( ?rf_label) = "Common Sense Reasoning") ?data orkgc:P30 ?rf; orkgc:P31 ?cont. ?cont orkgc:HAS_BENCHMARK ?benchmark; orkgc:P32 ?problem. ?problem rdfs:label ?problem_lbl. }</s>
512
<s> [INST]input (English text): Can you list benchmarked problems in the area of Semantic Web?
 output (Sparql query): [/INST]SELECT DISTINCT ?problem ?problem_lbl WH